In [9]:
import os
import uuid
import pickle

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [50]:
year = 2021
month = 2
taxi_type = 'green'
input_file = f"./data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
# input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'/output/{taxi_type}/{year:04d}-{month:02d}.parquet'

RUN_ID = os.getenv('RUN_ID', '8bb01fbcd9ab42ae9ba041b3d28788ad')

In [30]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df

def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [59]:
def load_model(run_id):
    #logged_model = f's3://mlflow-models-alexey/1/{RUN_ID}/artifacts/model'
    # /mlartifacts/1/
    logged_model = f'./mlartifacts/1/{RUN_ID}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id

    
    df_result.to_parquet(output_file, index=False)

    return df_result

In [60]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,3b5117aa-015d-42c5-a0fc-fbc1093c1c91,2021-02-01 00:34:03,130,205,17.916667,18.168946,-0.252279,8bb01fbcd9ab42ae9ba041b3d28788ad
1,a393e0c4-bc79-4326-8363-5a350e109b09,2021-02-01 00:04:00,152,244,6.500000,7.290331,-0.790331,8bb01fbcd9ab42ae9ba041b3d28788ad
2,a56fbfcf-8985-4678-995d-2f37319798dd,2021-02-01 00:18:51,152,48,15.250000,21.278875,-6.028875,8bb01fbcd9ab42ae9ba041b3d28788ad
3,c0426d4f-5474-4b1e-acce-8d07f106c7ae,2021-02-01 00:53:27,152,241,18.233333,25.008761,-6.775428,8bb01fbcd9ab42ae9ba041b3d28788ad
4,3334054d-49e9-47fc-9271-161ee6796d7e,2021-02-01 00:57:46,75,42,8.966667,9.380744,-0.414077,8bb01fbcd9ab42ae9ba041b3d28788ad
...,...,...,...,...,...,...,...,...
64567,a72a8cef-d158-41eb-ab88-fdb2eeae88cc,2021-02-28 22:19:00,129,7,10.000000,14.549670,-4.549670,8bb01fbcd9ab42ae9ba041b3d28788ad
64568,8f8c2481-5d2d-4d56-ac6f-2a80f13e8971,2021-02-28 23:18:00,116,166,9.000000,9.886653,-0.886653,8bb01fbcd9ab42ae9ba041b3d28788ad
64569,5d173864-6283-4f29-933c-bae451a8f8e8,2021-02-28 23:44:00,74,151,14.000000,13.597153,0.402847,8bb01fbcd9ab42ae9ba041b3d28788ad
64570,94b45844-dda2-4ed7-b563-cc450474e7c8,2021-02-28 23:07:00,42,42,7.000000,6.705995,0.294005,8bb01fbcd9ab42ae9ba041b3d28788ad


In [54]:
!ls output/green/

2021-02.parquet
